이번에는 자연어처리에서 빼놓을 수 없는 개념인 "토큰화(Tokenization)"와 "품사 태깅(POS Tagging)"에 대해 살펴본다.

In [37]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
# nltk.download('all') # 처음 실행 시 주석을 제거하고 실행

#### train 데이터 불러오기

In [38]:
df = pd.read_csv('./Data/train.csv')
df.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [39]:
df.isna().sum() # 결측치 확인

id          0
document    0
label       0
dtype: int64

##### 학습 / 검증셋 분리하기

In [40]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스 초기화
val.reset_index(inplace=True)

train # train 셋 확인

,index,id,document,label
0,1059,1060,주변에 적은 만들지 말자라는 교훈을 얻었다.,1
1,2039,2040,너무 허무하다.. 이거찍은 배우들이 불쌍함..,0
2,2283,2284,볼때마다 여운이 진하게남는 영화,1
3,1303,1304,2005년 영화라고 믿기지 않아. 양조위가 아깝다. 최여진 이뻐서 3점.,0
4,2918,2919,실패작 그 이상 그 이하도 아님 관객 조롱하는 영화,0
...,...,...,...,...
3745,181,182,한효주는 역시 연기의 신이다.매력이 넘쳐.,1
3746,286,287,아 궁금해서 봤는데 시간 ㅣ아깝다 아 내시간,0
3747,4918,4919,일본 야애니동자씨리즈를 표절하고 있다 표절좀하지마,0
3748,1701,1702,드라마 너무 재밌어요ㅠㅠㅠ오늘 방송도 기대!,1


##### 데이터 정제하기
document 컬럼의 리뷰들을 전처리해준다.<br>
현재 리뷰는 다양한 특수문자와 가장 먼저 한글과 공백만 남긴 뒤 preprocessed 라는 컬럼으로 저장

In [41]:
train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
train[:5] # 데이터 확인

,index,id,document,label,preprocessed
0,1059,1060,주변에 적은 만들지 말자라는 교훈을 얻었다.,1,주변에 적은 만들지 말자라는 교훈을 얻었다
1,2039,2040,너무 허무하다.. 이거찍은 배우들이 불쌍함..,0,너무 허무하다 이거찍은 배우들이 불쌍함
2,2283,2284,볼때마다 여운이 진하게남는 영화,1,볼때마다 여운이 진하게남는 영화
3,1303,1304,2005년 영화라고 믿기지 않아. 양조위가 아깝다. 최여진 이뻐서 3점.,0,년 영화라고 믿기지 않아 양조위가 아깝다 최여진 이뻐서 점
4,2918,2919,실패작 그 이상 그 이하도 아님 관객 조롱하는 영화,0,실패작 그 이상 그 이하도 아님 관객 조롱하는 영화


추가적으로 혹시 모를 다중공백도 제거해준다.

In [42]:
train['preprocessed'] = train['preprocessed'].str.replace(" +", " ") # 다중 공백 제거
train[:5]

,index,id,document,label,preprocessed
0,1059,1060,주변에 적은 만들지 말자라는 교훈을 얻었다.,1,주변에 적은 만들지 말자라는 교훈을 얻었다
1,2039,2040,너무 허무하다.. 이거찍은 배우들이 불쌍함..,0,너무 허무하다 이거찍은 배우들이 불쌍함
2,2283,2284,볼때마다 여운이 진하게남는 영화,1,볼때마다 여운이 진하게남는 영화
3,1303,1304,2005년 영화라고 믿기지 않아. 양조위가 아깝다. 최여진 이뻐서 3점.,0,년 영화라고 믿기지 않아 양조위가 아깝다 최여진 이뻐서 점
4,2918,2919,실패작 그 이상 그 이하도 아님 관객 조롱하는 영화,0,실패작 그 이상 그 이하도 아님 관객 조롱하는 영화


##### 토큰화
토큰화란?<br>
토큰화(Tokenization)란 말뭉치(Corpus)를 주어진 단위(Token)로 나누는 과정을 의미함<br>
따라서 다음과 같이 크게 두 종류로 구분해볼 수 있다.
1. 문장 토큰화
2. 단어 토큰화<br>
<div>먼저 말뭉치를 문장 단위로 나누는 문장 토큰화(sentence tokenization)의 예시를 살펴본다</div>

In [43]:
text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon." # 예시 문장을 정의함

In [44]:
from nltk.tokenize import sent_tokenize
print('문장 토큰화 결과 ==>', sent_tokenize(text))

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']


분리 기준으로 "!", "?", "." 등을 설정할 필요 없이 문장 단위로 말뭉치가 분리된다.<br>
다음으로 단어 단위로 나누는 단어 토큰화(word tokenization)의 예시를 살펴본다.

In [45]:
from nltk.tokenize import word_tokenize
print('단어 토큰화 결과 ==>', word_tokenize(text))

단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


단어 단위로 깔끔하게 분리됨. "!", "," 등의 구두점도 하나의 단어로 취급됨

##### 한국어에서의 토큰화
한국어는 영어와 달리 형태소(뜻을 가진 말의 최소 단위)간의 결합으로 어절이 이루어진다. ex) 철수 -가 책 -을 읽- -었- -다<br>
따라서 한국어 전용 형태소 분석기가 많이 존재하는데, 이번에는 koNLPy라는 패키지의 Okt라는 형태소 분석기를 사용함

In [46]:
from konlpy.tag import Okt

han_sentence = '오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ'
okt = Okt() # 인스턴스 할당
print('한국어 형태소 분석 결과(어간 추출X ==>', okt.morphs(han_sentence, stem=False)) # 형태소 단위로 분리
print('한국어 형태소 분석 결과(어간 추출O ==>', okt.morphs(han_sentence, stem=True)) # 형태소 단위로 분리 후 어간 추출

한국어 형태소 분석 결과(어간 추출X ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


stem=True 설정한 두 번째 결과를 보면 "해볼까" -> "해보다", "힘내서" -> "힘내다", "됩시다" -> "되다" 와 같이 각 형태소의 어간을 추출해준 것을 확인할 수 있다.<br>
필요에 따라 선택적으로 사용하면 좋을 것이다.<br>
이제 영화 리뷰 데이터에서 같은 방법으로 형태소 분석을 한다.

In [47]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내온다.
    tokens = okt.morphs(sentence, stem=True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해줌)
    tokenize = ' '.join(tokens) # tokens라는 리스트 안에 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가
train['tokenized_stem'] = pd.DataFrame(tokenized) # 리스트를 데이터프레이임으로 변환해 tokenized_stem라는 컬럼명으로 추가

train.head()

,index,id,document,label,preprocessed,tokenized_stem
0,1059,1060,주변에 적은 만들지 말자라는 교훈을 얻었다.,1,주변에 적은 만들지 말자라는 교훈을 얻었다,주변 에 적다 만들다 말 자 라는 교훈 을 얻다
1,2039,2040,너무 허무하다.. 이거찍은 배우들이 불쌍함..,0,너무 허무하다 이거찍은 배우들이 불쌍함,너무 허무하다 이 거 찍다 배우 들 이 불쌍하다
2,2283,2284,볼때마다 여운이 진하게남는 영화,1,볼때마다 여운이 진하게남는 영화,볼때 마다 여운 이 진하다 남다 영화
3,1303,1304,2005년 영화라고 믿기지 않아. 양조위가 아깝다. 최여진 이뻐서 3점.,0,년 영화라고 믿기지 않아 양조위가 아깝다 최여진 이뻐서 점,년 영화 라고 믿다 않다 양조위 가 아깝다 최여진 이쁘다 점
4,2918,2919,실패작 그 이상 그 이하도 아님 관객 조롱하는 영화,0,실패작 그 이상 그 이하도 아님 관객 조롱하는 영화,실패 작 그 이상 그 이하 도 아니다 관객 조롱 하다 영화


형태소 분석 및 어간 추출을완료한 "tokenized" 컬럼이 추가되었음<br>
<span style="color: gray">어렵다...</span>

##### 품사 태깅(POS Tagging)
품사 태깅이란 주어진 텍스트를 형태소 단위로 나눈 뒤, 각 형태소에 해당 품사를 태깅하여 리스트화 하는 것

In [48]:
# 품사 태깅
print(okt.pos('오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ'))

# 오 쩐다. 이렇게 태깅이 되는구나

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재밌는', 'Adjective'), ('코딩', 'Noun'), ('을', 'Josa'), ('해볼까', 'Verb'), ('?', 'Punctuation'), ('같이', 'Adverb'), ('힘내서', 'Verb'), ('자연어', 'Noun'), ('처리', 'Noun'), ('고수', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('됩시다', 'Verb'), ('!', 'Punctuation'), ('ㅎㅎ', 'KoreanParticle')]


In [49]:
print(okt.pos('이것은 1점이 아니다 11점을 주고싶은 내 간절한 마음이다'))

[('이', 'Determiner'), ('것', 'Noun'), ('은', 'Josa'), ('1', 'Number'), ('점', 'Noun'), ('이', 'Josa'), ('아니다', 'Adjective'), ('11', 'Number'), ('점', 'Noun'), ('을', 'Josa'), ('주고싶은', 'Verb'), ('내', 'Noun'), ('간절한', 'Adjective'), ('마음', 'Noun'), ('이다', 'Josa')]


한 예시를 살펴보니 Noun, Josa, Adverb, Adjective, Verb, Suffix, Punctuation, KoreanParticle 등으로 품사가 태깅되어 하나의 리스트 형태로 반환된다.<br>
태그되는 품사의 종류는 더 다양하니 직접 다양한 문장을 이용해 공부해보자<br>
okt의 pos 함수만을 이용해 토큰화와 품사 태깅을 한번에 할 수 있다.<br>
품사 태깅은 꼭 필요한 품사(ex. 명사)를 추출할 때 유용하게 쓰일 수 있다.<br>
바로 명사를 추출해보자

In [50]:
print(okt.nouns('오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ'))

['오늘', '코딩', '자연어', '처리', '고수']


okt의 nouns 함수를 사용해 간편하게 명사만을 추출했다.<br>
이렇게 간단하게 명사만을 추출할수도 있지만, 영화 리뷰는 명사만으로 긍정 / 부정을 판단하기 어렵기 때문에<br>
우선 임의로 명사, 동사, 형용사, 부사를 추출하여 사용한다.(이 부분은 직접 고민하고 수정해보라)

In [51]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in train['document']:  # 리뷰들을 하나씩 가져온다.
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받는다.
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join
    main_pos.append(main_words_str) # 선택한 형대소들로 이루어진 문자열을 최종 리스트에 추가
train['main_pos'] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가

train.head()

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,1059,1060,주변에 적은 만들지 말자라는 교훈을 얻었다.,1,주변에 적은 만들지 말자라는 교훈을 얻었다,주변 에 적다 만들다 말 자 라는 교훈 을 얻다,주변 적은 만들지 말 교훈 얻었다
1,2039,2040,너무 허무하다.. 이거찍은 배우들이 불쌍함..,0,너무 허무하다 이거찍은 배우들이 불쌍함,너무 허무하다 이 거 찍다 배우 들 이 불쌍하다,너무 허무하다 거 찍은 배우 불쌍함
2,2283,2284,볼때마다 여운이 진하게남는 영화,1,볼때마다 여운이 진하게남는 영화,볼때 마다 여운 이 진하다 남다 영화,볼때 여운 진하게 남는 영화
3,1303,1304,2005년 영화라고 믿기지 않아. 양조위가 아깝다. 최여진 이뻐서 3점.,0,년 영화라고 믿기지 않아 양조위가 아깝다 최여진 이뻐서 점,년 영화 라고 믿다 않다 양조위 가 아깝다 최여진 이쁘다 점,영화 믿기지 않아 양조위 아깝다 최여진 이뻐서 점
4,2918,2919,실패작 그 이상 그 이하도 아님 관객 조롱하는 영화,0,실패작 그 이상 그 이하도 아님 관객 조롱하는 영화,실패 작 그 이상 그 이하 도 아니다 관객 조롱 하다 영화,실패 작 그 이상 그 이하 아님 관객 조롱 하는 영화


이제 총 세 개의 전처리된 컬럼이 생성되었다.<br>
어떤 전처리 과정이 성능이 가장 잘 나올지는 미지수이다.<br>
직접 다양한 시도를 통해 죄적의 전처리 프로세스를 찾아보자<br>
이번에는 마지막에 생성한 main_pos 컬럼을 이용해 모델 학습을 진행한다.

##### 벡터화
CountVectorizer 사용

In [52]:
x_train = train.main_pos
y_train = train.label

In [53]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(x_train)
x_train_vec = vectorizer.transform(x_train)

##### 모델 학습

In [54]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train_vec, y_train)

LogisticRegression()

##### 검증셋으로 모델 성능 검증
먼저 검증셋 val에 train셋과 동일한 전처리 과정을 거침

In [55]:
# preprocessed
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
val['preporcessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem=True)
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val['tokenized_stem'] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val['main_pos'] = pd.DataFrame(main_pos)

val.head()

,index,id,document,label,preprocessed,preporcessed,tokenized_stem,main_pos
0,3793,3794,짧았지만 가장 강한 인상을 주었던 정우성.,1,짧았지만 가장 강한 인상을 주었던 정우성,짧았지만 가장 강한 인상을 주었던 정우성,짧다 가장 강하다 인상 을 주다 정우성,짧았지만 가장 강한 인상 주었던 정우성
1,1172,1173,법은강한자에게만 평등하고정의는 없다,1,법은강한자에게만 평등하고정의는 없다,법은강한자에게만 평등하고정의는 없다,법 은 강 한자 에게만 평등하다 정의 는 없다,법 강 한자 평등하고 정의 없다
2,1747,1748,"캐리는 재수없지만, 재미있는건 어쩔수없지. 사만다짱",1,캐리는 재수없지만 재미있는건 어쩔수없지 사만다짱,캐리는 재수없지만 재미있는건 어쩔수없지 사만다짱,캐리 는 재수없다 재미있다 어쩔 수 없다 사만 다 짱,캐리 재수없지만 재미있는건 수 없지 사만 다 짱
3,795,796,남자들의 로망 문근영의 리즈시절ㅎㅎ 보기만해도 미소가 절로 나오는 영화ㅎㅎ,1,남자들의 로망 문근영의 리즈시절ㅎㅎ 보기만해도 미소가 절로 나오는 영화ㅎㅎ,남자들의 로망 문근영의 리즈시절ㅎㅎ 보기만해도 미소가 절로 나오는 영화ㅎㅎ,남자 들 의 로망 문근영 의 리즈시절 ㅎㅎ 보기 만해 도 미소 가 절로 나오다 영화 ㅎㅎ,남자 로망 문근영 리즈시절 보기 만해 미소 절로 나오는 영화
4,3470,3471,재밌어요 앞으로도 기대할께요^^,1,재밌어요 앞으로도 기대할께요,재밌어요 앞으로도 기대할께요,재밌다 앞 으로도 기대하다,재밌어요 앞 기대할께요


학습한 모델로 예측할 x를 벡터화 해준다.

In [56]:
x_val = val.main_pos
y_val = val.label

x_val_vec = vectorizer.transform(x_val)

In [57]:
# run model
y_pred = model.predict(x_val_vec)
print(y_pred)
# 0: negative / 1: positive

[1 0 0 ... 0 1 1]


In [58]:
from sklearn import metrics
print('accuarcy =', metrics.accuracy_score(y_val, y_pred))

accuarcy = 0.7968


train 셋으로 학습한 모델을 사용해 validation 셋을 예측해본 결과 0.8344의 정확도를 얻음<br>
다양한 전처리 기법과 모델들을 사용하여 성능을 높여보자

##### test.csv 분류하기
이번엔 대회에서 주어진 정답이 없는 test 데이터의 라벨을 예측해봄

In [59]:
test = pd.read_csv('./Data/test.csv')
test.head()

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다


In [60]:
test.isna().sum()

id          0
document    0
dtype: int64

In [61]:
# preprocessed
test['preprocessed'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
test['preprocessed'] = test['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in test['preprocessed']:
    tokens = okt.morphs(sentence, stem=True)
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
test['tokenized_stem'] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test['main_pos'] = pd.DataFrame(main_pos)

test.head()

,id,document,preprocessed,tokenized_stem,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,,,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화입니다 잊을 수 없는,멋있다 영화 이다 잊다 수 없다,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동적이네요 펑펑 울었습니다,너무 감동 적다 펑펑 울다,너무 감동 적이네요 펑펑 울었습니다


In [62]:
x_test = test.main_pos
x_test_vec = vectorizer.transform(x_test)
pred_test = model.predict(x_test_vec)
print(pred_test)

[0 1 0 ... 1 0 1]
